## Emotion Classification with BERT

### Installing dependencies

In [51]:
!pip install -q transformers datasets torch accelerate tqdm evaluate safetensors

### Setting environment variables for the Hugging Face cache.

In [1]:
import os
os.environ["HF_HOME"] = "hf_cache"
os.environ['HF_EVALUATE_OFFLINE'] = '1'

### Loading the Hugging Face emotion dataset

In [2]:
from datasets import load_dataset

# Load the 'split' configuration of the emotion dataset
raw_datasets  = load_dataset("dair-ai/emotion", name="split")

c:\Users\priks\anaconda3\envs\t\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using the latest cached version of the dataset since dair-ai/emotion couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'split' at hf_cache\datasets\dair-ai___emotion\split\0.0.0\cab853a1dbdf4c42c2b3ef2173804746df8825fe (last modified on Sun Oct 20 22:36:23 2024).


In [3]:
raw_datasets["train"][0]

{'text': 'i didnt feel humiliated', 'label': 0}

In [4]:
raw_datasets["train"].features

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['sadness', 'joy', 'love', 'anger', 'fear', 'surprise'], id=None)}

### Tokenization

In [5]:
from transformers import AutoTokenizer

# Load the BERT tokenizer
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# Tokenize function
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Tokenize the dataset
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)


In [6]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
})

In [7]:
print(tokenized_datasets["train"][0])

{'text': 'i didnt feel humiliated', 'label': 0, 'input_ids': [101, 1045, 2134, 2102, 2514, 26608, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [8]:
tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

In [9]:
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# train_dataloader = DataLoader(
#     tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
# )
# eval_dataloader = DataLoader(
#     tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator
# )

# for batch in train_dataloader:
#     break
# {k: v.shape for k, v in batch.items()}

### Load the model

In [11]:
from transformers import AdamW, AutoModelForSequenceClassification

# Load the model
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=6)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Define compute metrics

In [12]:
import evaluate
import numpy as np

accuracy_metric = evaluate.load("accuracy")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    precision = precision_metric.compute(predictions=predictions, references=labels, average="macro")
    recall = recall_metric.compute(predictions=predictions, references=labels, average="macro")
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="macro")

    return {
        "accuracy": accuracy["accuracy"],
        "precision": precision["precision"],
        "recall": recall["recall"],
        "f1": f1["f1"]
    }



Using the latest cached version of the module from hf_cache\modules\evaluate_modules\metrics\evaluate-metric--accuracy\f887c0aab52c2d38e1f8a215681126379eca617f96c447638f751434e8e65b14 (last modified on Sun Oct 20 15:39:45 2024) since it couldn't be found locally at evaluate-metric--accuracy, or remotely on the Hugging Face Hub.
Using the latest cached version of the module from hf_cache\modules\evaluate_modules\metrics\evaluate-metric--precision\4e7f439a346715f68500ce6f2be82bf3272abd3f20bdafd203a2c4f85b61dd5f (last modified on Sun Oct 20 15:48:15 2024) since it couldn't be found locally at evaluate-metric--precision, or remotely on the Hugging Face Hub.
Using the latest cached version of the module from hf_cache\modules\evaluate_modules\metrics\evaluate-metric--recall\e40e6e98d18ff3f210f4d0b26fa721bfaa80704b1fdf890fa551cfabf94fc185 (last modified on Sun Oct 20 15:56:45 2024) since it couldn't be found locally at evaluate-metric--recall, or remotely on the Hugging Face Hub.
Using the la

### Defining TrainingArguments and Trainer

In [13]:
from transformers import Trainer
from transformers import TrainingArguments
from transformers import EarlyStoppingCallback

training_args = TrainingArguments(
    output_dir="trainer",
    evaluation_strategy="epoch",  # Evaluate model at the end of each epoch
    learning_rate=2e-5,  # Initial learning rate for the AdamW optimizer
    per_device_train_batch_size=16,  # Batch size per device during training
    per_device_eval_batch_size=16,  # Batch size per device during evaluation
    num_train_epochs=20,  # Number of training epochs
    warmup_steps=500,  # Number of steps to warm up the learning rate
                        # Reason: Gradually increases the learning rate to prevent sudden large updates that could destabilize training in early stages.
    weight_decay=0.01,  # Strength of weight decay regularization
                        # Reason: Prevents overfitting by penalizing large weights in the model, commonly used in fine-tuning.
                        
    logging_steps=100,  # Log training progress every 100 steps
    save_strategy="epoch",  # Save checkpoints at the end of each epoch
    save_total_limit=3,  # Keep only the last 3 checkpoints to save disk space
    load_best_model_at_end=True,  # Load the best model after training is finished based on evaluation metrics
    metric_for_best_model="f1",  # Use F1 score to determine the best model
    greater_is_better=True,  # Higher F1 score is better
    fp16=True,  # Use mixed precision training (FP16) for faster training and reduced memory usage on NVIDIA GPUs
    dataloader_pin_memory=True,  # Pin data loader memory for faster GPU transfers
    gradient_accumulation_steps=2,  # Accumulate gradients over 2 steps before updating, effectively simulating a larger batch size
    max_grad_norm=1.0,  # Clip gradients to avoid exploding gradients, with a max norm of 1.0
    auto_find_batch_size=True  # Automatically adjust batch size to avoid out-of-memory errors
)


trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]  # Early stop if no improvement in 3 evals
)

c:\Users\priks\anaconda3\envs\t\lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


### Model Training

In [14]:
trainer.train()

  0%|          | 0/10000 [00:00<?, ?it/s]c:\Users\priks\anaconda3\envs\t\lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
  1%|          | 100/10000 [00:35<57:30,  2.87it/s]

{'loss': 1.7792, 'grad_norm': 4.088680744171143, 'learning_rate': 3.96e-06, 'epoch': 0.2}


  2%|▏         | 200/10000 [01:09<57:07,  2.86it/s]

{'loss': 1.5477, 'grad_norm': 10.588886260986328, 'learning_rate': 7.92e-06, 'epoch': 0.4}


  3%|▎         | 300/10000 [01:44<56:28,  2.86it/s]

...

                                                    
 35%|███▌      | 3500/10000 [21:10<37:41,  2.87it/s]

{'eval_loss': 0.25887352228164673, 'eval_accuracy': 0.9395, 'eval_precision': 0.9323470481226465, 'eval_recall': 0.9018501516898346, 'eval_f1': 0.9154533008896871, 'eval_runtime': 6.2253, 'eval_samples_per_second': 321.269, 'eval_steps_per_second': 20.079, 'epoch': 7.0}


 35%|███▌      | 3500/10000 [21:11<39:21,  2.75it/s]

{'train_runtime': 1271.509, 'train_samples_per_second': 251.669, 'train_steps_per_second': 7.865, 'train_loss': 0.25444636344909666, 'epoch': 7.0}


TrainOutput(global_step=3500, training_loss=0.25444636344909666, metrics={'train_runtime': 1271.509, 'train_samples_per_second': 251.669, 'train_steps_per_second': 7.865, 'total_flos': 2.9469496541184e+16, 'train_loss': 0.25444636344909666, 'epoch': 7.0})

### Saving the Model and Tokenizer

In [18]:
# Saving the Model and Tokenizer

import json
model_output_dir = "model"

trainer.save_model(model_output_dir)

tokenizer.save_pretrained(model_output_dir)

# Save the evaluation metrics as a JSON file
metrics_output_file = model_output_dir + "/metrics.json"
with open(metrics_output_file, "w") as f:
    json.dump(trainer.state.log_history, f)

In [21]:
import torch
from transformers import TrainingArguments

# Load the training arguments from the bin file
training_args = torch.load("model/training_args.bin")

# Convert to dictionary to view contents
training_args_dict = training_args.to_dict()

# Print the training arguments
for key, value in training_args_dict.items():
    print(f"{key}: {value}")



output_dir: trainer
overwrite_output_dir: False
do_train: False
do_eval: True
do_predict: False
eval_strategy: epoch
prediction_loss_only: False
per_device_train_batch_size: 16
per_device_eval_batch_size: 16
per_gpu_train_batch_size: None
per_gpu_eval_batch_size: None
gradient_accumulation_steps: 2
eval_accumulation_steps: None
eval_delay: 0
torch_empty_cache_steps: None
learning_rate: 2e-05
weight_decay: 0.01
adam_beta1: 0.9
adam_beta2: 0.999
adam_epsilon: 1e-08
max_grad_norm: 1.0
num_train_epochs: 20
max_steps: -1
lr_scheduler_type: linear
lr_scheduler_kwargs: {}
warmup_ratio: 0.0
warmup_steps: 500
log_level: passive
log_level_replica: warning
log_on_each_node: True
logging_dir: trainer\runs\Oct20_23-26-19_Prism
logging_strategy: steps
logging_first_step: False
logging_steps: 100
logging_nan_inf_filter: True
save_strategy: epoch
save_steps: 500
save_total_limit: 3
save_safetensors: True
save_on_each_node: False
save_only_model: False
restore_callback_states_from_checkpoint: False
no_

In [56]:
import json

# Function to filter out non-serializable objects
def custom_encoder(obj):
    if isinstance(obj, dict):
        return {k: v for k, v in obj.items() if isinstance(v, (int, float, str, bool, list, dict))}
    return str(obj)  # If it's an unsupported type, convert it to a string

# Assuming `training_args` is your TrainingArguments object
training_args_dict = vars(training_args)

# Manually filter out non-serializable objects and save as JSON
with open("model/training_args.json", "w") as f:
    json.dump(training_args_dict, f, default=custom_encoder, indent=4)

print("Training arguments saved to training_args.json")


Training arguments saved to training_args.json


### Model Evaluation

In [26]:
from tqdm import tqdm

model_output_dir = "model"

# Load the saved model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained(model_output_dir)
tokenizer = AutoTokenizer.from_pretrained(model_output_dir)

# Prepare the test dataset
test_dataset = tokenized_datasets["test"]

# Create DataLoader for the test dataset
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=True, collate_fn=data_collator)

# Move model to the appropriate device (GPU if available, else CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Put the model in evaluation mode
model.eval()

# Initialize lists to store logits and true labels
logits_list = []
true_labels = []

# Run inference on the test dataset
with torch.no_grad():
    for batch in tqdm(test_dataloader):
        # Move batch to the same device as the model
        inputs = {k: v.to(device) for k, v in batch.items() if k != "labels"}
        labels = batch["labels"].to(device)

        # Forward pass
        outputs = model(**inputs)
        logits = outputs.logits

        # Store logits and true labels
        logits_list.append(logits.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Convert logits to a NumPy array for metric calculation
logits_array = np.concatenate(logits_list, axis=0)
true_labels_array = np.array(true_labels)

# Compute and print the metrics using your compute_metrics function
metrics = compute_metrics((logits_array, true_labels_array))

# Print the results
print("Test set evaluation metrics:")
print(metrics)

# Optionally, save the metrics to a file
import json
test_metrics_output_file = model_output_dir + "/test_metrics.json"
with open(test_metrics_output_file, "w") as f:
    json.dump(metrics, f)

100%|██████████| 125/125 [00:15<00:00,  8.01it/s]

Test set evaluation metrics:
{'accuracy': 0.918, 'precision': 0.8663272444541898, 'recall': 0.8756554808732849, 'f1': 0.8705580483226839}


In [58]:
from transformers import AutoConfig

# Load the model configuration
model_name = "model"
config = AutoConfig.from_pretrained(model_name)

# Set `id2label` and `label2id` dynamically from the dataset
config.id2label = {i: label for i, label in enumerate(raw_datasets["train"].features["label"].names)}
config.label2id = {label: i for i, label in enumerate(raw_datasets["train"].features["label"].names)}

# Save the updated configuration back to the model repository
config.save_pretrained(model_name)


In [45]:
from huggingface_hub import HfApi

repo_name = "bert-base-uncased-emotion"
repo_id = f"Prikshit7766/{repo_name}"
token = "" 
api = HfApi()
api.create_repo(repo_id=repo_id, private=False, token=token)



RepoUrl('https://huggingface.co/Prikshit7766/bert-base-uncased-emotion', endpoint='https://huggingface.co', repo_type='model', repo_id='Prikshit7766/bert-base-uncased-emotion')

In [72]:
import os
from huggingface_hub import HfApi

# Initialize API and repository details
repo_name = "bert-base-uncased-emotion"
repo_id = f"Prikshit7766/{repo_name}"
model_dir = "./model"  # Path to the model directory where all your files are located

# Initialize HfApi
api = HfApi()

# Loop through all files in the model directory
for root, dirs, files in os.walk(model_dir):
    for file in files:
        file_path = os.path.join(root, file)  # Full path to the file
        relative_path_in_repo = file  # This is the path that will be used in the repo
        
        # Upload the file to Hugging Face Hub
        api.upload_file(
            path_or_fileobj=file_path,
            path_in_repo=relative_path_in_repo,
            repo_id=repo_id,
            token=token
        )
        print(f"Uploaded {file_path} to Hugging Face repository {repo_id}")


Uploaded ./model\config.json to Hugging Face repository Prikshit7766/bert-base-uncased-emotion
Uploaded ./model\metrics.json to Hugging Face repository Prikshit7766/bert-base-uncased-emotion
Uploaded ./model\model.safetensors to Hugging Face repository Prikshit7766/bert-base-uncased-emotion
Uploaded ./model\README.md to Hugging Face repository Prikshit7766/bert-base-uncased-emotion
Uploaded ./model\special_tokens_map.json to Hugging Face repository Prikshit7766/bert-base-uncased-emotion
Uploaded ./model\test_metrics.json to Hugging Face repository Prikshit7766/bert-base-uncased-emotion
Uploaded ./model\tokenizer.json to Hugging Face repository Prikshit7766/bert-base-uncased-emotion
Uploaded ./model\tokenizer_config.json to Hugging Face repository Prikshit7766/bert-base-uncased-emotion
Uploaded ./model\training_args.bin to Hugging Face repository Prikshit7766/bert-base-uncased-emotion
Uploaded ./model\training_args.json to Hugging Face repository Prikshit7766/bert-base-uncased-emotion
U